In [68]:
#setup libraries
import numpy as np
import pandas as pd
import os, re, glob, sys
from skimage import io, filters, util, segmentation, morphology, measure, restoration, exposure
import matplotlib.pyplot as plt
#import imageio
from cellpose import models, plot
from pystackreg import StackReg
from scipy import stats, spatial
from PIL import Image, ImageSequence
import tifffile

In [69]:
pipeline_name = "CK" #cellpose and KIT tracking
ch1_name = 'NR'
ch2_name = 'CC'
data_path = '/home/exacloud/gscratch/HeiserLab/images/'
#data_path = '/Users/dane/Documents/CellTrackingProjects/AU565/images/'
plateID = 'AU02001'
#plateID = sys.argv[1]
well_index = 2
#well_index = int(sys.argv[2])

output_path = os.path.join(data_path+plateID,"Analysis",pipeline_name,"intermediate_files/")
transformation_path = os.path.join(output_path,"transformations")
tracking_path = os.path.join(output_path,'tracking/')

#Only use subdirectories within the selected well
well_directory = sorted(glob.glob(os.path.join(data_path+plateID,"[A-Z][1-9]")))[well_index-1]
well = re.findall("[A-Z][1-9]$",well_directory)[0]
subdirectories = sorted(glob.glob(os.path.join(well_directory,"field_[1-9]")))

#### Register the image stacks
If there is a registered red channel stack skip this step, otherwise:  
Load the red, green and phase images  
Delete images from any time slice that does not have a complete set of images  
Rescale the fluorescent images from 12 to 8 bits  
Calculate the transformations needed to register the red stack, correcting the translation only  
Store the registration transformations  
Use the transformations to register all three stacks  
Save the registered stacks as 16 bit images  

TODO: See if it's possible to not rescale the images  

In [70]:
flourescent_scaler = 255/4095 #rescale from 12 to 8 bits

for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    reg_filename = os.path.join(output_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    # Only process the field-level image files if there is no registered stack
    if not os.path.exists(reg_filename):
        print("registering R stack in "+subdir)
        #load and prepare red, green and phase channels. Scale for 8 bits but these are uint16 data types
        r_data_paths = glob.glob(os.path.join(subdir,"*_R_*m.tif"))
        r_time_slices = set()
        for data_paths in r_data_paths:
            r_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        g_data_paths = glob.glob(os.path.join(subdir,"*_G_*m.tif"))
        g_time_slices = set()
        for data_paths in g_data_paths:
            g_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        p_data_paths = glob.glob(os.path.join(subdir,"*_P_*m.tif"))
        p_time_slices = set()
        for data_paths in p_data_paths:
            p_time_slices.add(re.findall("..d..h..m", data_paths)[0])
        complete_time_slices = r_time_slices & g_time_slices & p_time_slices
        r_data_paths_c = []
        g_data_paths_c = []
        p_data_paths_c = []
        for time_slice in complete_time_slices:
            r_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_R_"+well+"_"+field_num+"_"+time_slice+".tif"))
            g_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_G_"+well+"_"+field_num+"_"+time_slice+".tif"))
            p_data_paths_c.append(os.path.join(data_path+plateID,well,field,plateID+"_P_"+well+"_"+field_num+"_"+time_slice+".tif"))
        img_r_ic = io.imread_collection(r_data_paths_c) # 3 dimensions : frames x width x height
        img_rs = np.stack(img_r_ic)*flourescent_scaler

        img_g_ic = io.imread_collection(g_data_paths_c) # 3 dimensions : frames x width x height
        img_gs = np.stack(img_g_ic)*flourescent_scaler

        img_p_ic = io.imread_collection(p_data_paths_c) # 3 dimensions : frames x width x height
        img_ps = np.stack(img_p_ic)
        
        #register the R stack using transformation
        sr = StackReg(StackReg.TRANSLATION)
        # register each frame to the previous (already registered) one
        tmats = sr.register_stack(img_rs, reference='previous')
        if not os.path.exists(transformation_path):
            os.makedirs(transformation_path)
        np.save(os.path.join(transformation_path,plateID+"_"+well+"_"+field+"_transformation_matrices.npy"), tmats)
        
        # transform stack using the tmats loaded from file
        img_rs_reg = sr.transform_stack(img_rs, tmats = tmats)
        img_gs_reg = sr.transform_stack(img_gs, tmats = tmats)
        img_ps_reg = sr.transform_stack(img_ps, tmats = tmats)
        img_c_reg = np.stack((img_rs_reg.astype(np.int16), img_ps_reg.astype(np.int16)), axis = 1) #create tcxy 

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        print("saving "+reg_filename)
        io.imsave(reg_filename, img_rs_reg.astype(np.int16), plugin='tifffile', check_contrast=False)
        print("saving "+reg_filename.replace("_R_","_G_"))
        io.imsave(reg_filename.replace("_R_","_G_"), img_gs_reg.astype(np.int16), plugin='tifffile', check_contrast=False)
        print("saving "+reg_filename.replace("_R_","_P_"))
        io.imsave(reg_filename.replace("_R_","_P_"), img_ps_reg.astype(np.int16), plugin='tifffile', check_contrast=False)
        print("saving "+reg_filename.replace("_R_","_C_"))
        io.imsave(reg_filename.replace("_R_","_C_"), img_c_reg, plugin='tifffile', check_contrast=False)



#### Segment the phase  images using cellpose  
If mask files already exist, skip this step, otherwise:    
Load the registered phase images  
Segment each image and only keep masks greater than a selected area  
Save the mask files as an image sequence  
For compatibility with the tracking method, save the masks and the phase images as indivdual files  

In [71]:
diameter = 16
nuc_diameter = 13
flow_threshold = 0
nuc_flow_threshold = .7
mask_threshold=0
nuc_mask_threshold=0
min_size=75
nuc_min_size=75
resample = True
cyto_expansion = 5

#call cellpose on each image to segment the phase cytoplasm and nuclei
# DEFINE CELLPOSE MODELs
cell_model = models.Cellpose(gpu=True, model_type='cyto')
nuc_model = models.Cellpose(gpu=True, model_type='nuclei')

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# will use channel R = 1 as nuclear channel only
cell_channels = [2,1]
nuc_channels = [1,0]

for subdir in subdirectories:
    results = [] #collect results for one field in the well
    field = re.findall("field_[1-9]",subdir)[0]
    field_num = re.findall("[0-9]", field)[0]
    #reg_filename = os.path.join(output_path,plateID+"_R_"+well+"_"+field_num+"_reg_stack.tif")
    #Segment on C stack which has phase in green and nuclear in red
    reg_filename = os.path.join(output_path,plateID+"_C_"+well+"_"+field_num+"_reg_stack.tif")
    cell_mask_filename = reg_filename.replace("_reg_stack.tif", "_masks_stack.png")
    tracking_path = os.path.join(output_path,'tracking/')
    if not os.path.exists(tracking_path+well+"/"+field+"/masks/"):
            os.makedirs(tracking_path+well+"/"+field+"/masks/")
    if not os.path.exists(tracking_path+well+"/"+field+"/cyto_masks/"):
            os.makedirs(tracking_path+well+"/"+field+"/cyto_masks/")
    if not os.path.exists(tracking_path+well+"/"+field+"/nuc_masks/"):
            os.makedirs(tracking_path+well+"/"+field+"/nuc_masks/")
    if not os.path.exists(tracking_path+well+"/"+field+"/nuc_exp_masks/"):
            os.makedirs(tracking_path+well+"/"+field+"/nuc_exp_masks/")
    if not os.path.exists(tracking_path+well+"/"+field+"/reg"):
            os.makedirs(tracking_path+well+"/"+field+"/reg")
    if not os.path.exists(tracking_path+well+"/"+field+"/results"):
            os.makedirs(tracking_path+well+"/"+field+"/results")
    if not os.path.exists(tracking_path+well+"/"+field+"/filtered_masks"):
            os.makedirs(tracking_path+well+"/"+field+"/filtered_masks")

    if not os.path.exists(cell_mask_filename): #Only segment if no cell mask file
        img_stack = io.imread(reg_filename)
                         
        cell_mask_images = []
        cyto_mask_images = []
        nuc_mask_images = []
        nuc_exp_mask_images = []
        for i, image in enumerate(img_stack): 
            print("processing "+reg_filename+" index "+str(i))

            # create masks with cellpose 
            cell_masks, flows, styles, diams = cell_model.eval(image,
                                             diameter=diameter,
                                             flow_threshold=flow_threshold,
                                             mask_threshold=mask_threshold,
                                             channels=cell_channels,
                                             min_size=min_size,
                                             resample = resample)
       
            #Save mask image
            cell_mask_images.append(cell_masks)
            
             # create nuclear masks with cellpose 
            nuc_masks, flows, styles, diams = nuc_model.eval(image,
                                             diameter=nuc_diameter,
                                             flow_threshold=nuc_flow_threshold,
                                             mask_threshold=nuc_mask_threshold,
                                             channels=nuc_channels,
                                             min_size=nuc_min_size,
                                             resample = resample)
       
            #Save mask image
            nuc_mask_images.append(nuc_masks)
            
            #create a cyto only mask by zeroing out cell pixels that are in the nuclear masks
            nuc_logical_mask = nuc_masks == 0
            cyto_masks = cell_masks * nuc_logical_mask
            cyto_mask_images.append(cyto_masks)
            
            #create a ring around the neuclei
            nuclei_boundaries = segmentation.find_boundaries(nuc_masks, mode='thick')*nuc_masks
            nuc_exp_masks = segmentation.expand_labels(nuc_masks, cyto_expansion) - nuc_masks + nuclei_boundaries
            nuc_exp_mask_images.append(nuc_exp_masks)

       
        tifffile.imwrite(cell_mask_filename, np.array(cell_mask_images), imagej=True)
        tifffile.imwrite(cell_mask_filename.replace('masks', 'nuc_masks'), np.array(nuc_mask_images), imagej=True)
        tifffile.imwrite(cell_mask_filename.replace('masks', 'cyto_masks'), np.array(cyto_mask_images), imagej=True)
        tifffile.imwrite(cell_mask_filename.replace('masks', 'nuc_exp_masks'), np.array(nuc_exp_mask_images), imagej=True)

        # Open the cell mask stack:
        im = Image.open(cell_mask_filename)
 
        # create an index variable:
        i =0
        app = []
 
        # iterate over the cyto mask stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/masks/"+"mask%03.d.tif"%i)
            i = i + 1
            
         # Open the nuclear mask stack:
        im = Image.open(cell_mask_filename.replace('masks', 'nuc_masks'))
 
        # create an index variable:
        i =0
        app = []
 
        # iterate over the nuclear mask stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/nuc_masks/"+"nuc_mask%03.d.tif"%i)
            i = i + 1
            
         # Open the cyto mask stack:
        im = Image.open(cell_mask_filename.replace('masks', 'cyto_masks'))
 
        # create an index variable:
        i =0
        app = []
 
        # iterate over the nuclear mask stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/cyto_masks/"+"cyto_mask%03.d.tif"%i)
            i = i + 1
           
         # Open the nucelear expansion mask stack:
        im = Image.open(cell_mask_filename.replace('masks', 'nuc_exp_masks'))
 
        # create an index variable:
        i =0
        app = []
 
        # iterate over the nuclear mask stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/nuc_exp_masks/"+"nuc_exp_mask%03.d.tif"%i)
            i = i + 1
 
        # Open the registered intensity stack: use the phase images
        im = Image.open(reg_filename.replace("_C_", "_P_"))
 
        # create an index variable:
        i = 0
        app = []
 
        # iterate over the registered phase stack and save each frame to disk:
        for fr in ImageSequence.Iterator(im):
            app.append(fr)
            fr.save(tracking_path+well+"/"+field+"/reg/"+"t%03.d.tif"%i)
            i = i + 1 

processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 0
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 1
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 2
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 3
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 4
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 5
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_1_reg_stack.tif index 6
processing /home/exacloud/gscratch/HeiserLab/images/AU02001/Analysis/CK/intermediate_files/AU02001_C_A2_

#### Track the nuclei  
Use the KIT-Loeffler tracking method to track the masks and the nuclei    
For now, use the string below to run from the tracking script from the command line  
The tracking output includes masks with new label values and a res_track.txt file as described below  

In [72]:
for subdir in subdirectories: #track all fileds in the well
    field = re.findall("field_[1-9]",subdir)[0]
    #Condition on the tracking output file res_tracks.txt existing
    if not os.path.exists(tracking_path+well+"/"+str(field)+"/results/res_track.txt"): #Only segment if no mask file
        cmd = "python -m run_tracking --image_path "+ tracking_path+well+"/"+str(field)+"/reg/ --segmentation_path "+tracking_path+well+"/"+str(field)+"/masks/ --results_path "+tracking_path+well+"/"+str(field)+"/results --delta_t 3 --default_roi_size 2"
        returned_value = os.system(cmd)  # returns the exit code in unix
        print('returned value from command line tracking with KIT:', returned_value)


####################
time point: 0
####################
####################
time point: 1
####################
####################
time point: 2
####################
####################
time point: 3
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 3216.05
####################
time point: 4
####################
####################
time point: 5
####################
####################
time point: 6
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 9813.18
####################
time point: 7
####################
####################
time point: 8
####################
####################
time point: 9
####################
Add vertices to graph
Add sink and source vertex to graph
Set up constraints
Add Equations
Optimize
Optimal objective: 10980.3
####################
time point: 10
####################
##

#### Identify cells
Read in the tracking results  
res_track.txt - A text file representing an acyclic graph for the whole image sequence. Every line corresponds to a single track that is encoded by four numbers separated by a space:  
L B E P where  
L - a unique label of the track (label of markers, 16-bit positive value)  
B - a zero-based temporal index of the frame in which the track begins  
E - a zero-based temporal index of the frame in which the track ends  
P - label of the parent track (0 is used when no parent is defined)


Filter the track objects keeping the parents and those with a minimum track length and save the results to tracks.csv    

Create a new file tracks.csv with the following columns:  
label - a unique label of the track (label of markers, 16-bit positive value)  
begins - a zero-based temporal index of the frame in which the track begins  
ends - a zero-based temporal index of the frame in which the track ends  
parent - label of the parent track (0 is used when no parent is defined)  
length - The number of frames that the cell is identified in  
plateID - Character string of the plate's ID such as AU02001  
well - Character string of the well such as A1  
field - Integer of the image field within the well  

In [73]:
#set filter parameters
min_track_length = 3
#loop through the results from each segmented field
for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    res_filename = os.path.join(output_path,"tracking",well,field,"results","res_track.txt")
    res_flt_filename = res_filename.replace("res_track.txt","tracks.csv")
        #If filtered results do not exist, read in the res_track.txt file for the current field
    if not os.path.exists(res_flt_filename):
        tracks = pd.read_csv(res_filename, sep=" ", header=None)
        tracks.columns = ["label", "begins", "ends", "parent"]
        tracks['length'] = tracks.ends - tracks.begins + 1
        #check if object is a parent
        tracks["is_parent"] = tracks['label'].isin(tracks['parent'])
        tracks['plateID'] = plateID
        tracks['well'] = well
        tracks['field'] = field.replace("field_","")

        #Filter using the filter parameters
        #remove short tracks that are not parents
        tracks_flt = tracks.query('length >= @min_track_length or is_parent')
        ##remove any track that appears after the first frame and doesn't have a parent
        #tracks_flt = tracks_flt.query('not (begins > 1 & parent == 0)')
        #write out the res_flt_track.txt file
        tracks_flt.to_csv(res_flt_filename,index=False) 

#### Filter masks to only tracked cells  
Use the filtered tracks to remove masks for non-cell objects  
Save the filtered masks as individual image files in filtered_masks directory   

In [74]:
#loop through the fields in the well
for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    res_flt_filename = os.path.join(output_path,"tracking",well,field,"results","tracks.csv")
    mask_track_path = os.path.join(output_path,"tracking",well,field,"results")
    filtered_tracks_file_count = len(glob.glob(mask_track_path.replace("results","filtered_masks")+"/mask*"))
    if not filtered_tracks_file_count > 1:
       #read in the tracks file for this field
        tracks = pd.read_csv(res_flt_filename) 
        #loop through the mask images in the field
        tracked_mask_filenames = sorted(glob.glob(mask_track_path+"/mask*"))
        # iterate over the mask files
        for fn in tracked_mask_filenames:
            #read in the mask image
            im = io.imread(fn)
            #replace any label that's not a cell with a 0 value
            cell_labels = np.array([x if x in tracks.label.to_numpy()
                                       else 0 for x in range(0, im.max()+1)])
            im_filtered = cell_labels[im]
            io.imsave(fn.replace("results","filtered_masks"), im_filtered.astype(np.int16), plugin='tifffile', check_contrast=False)
        

#### Get excel metadata file  
If this file does not exists, cretae a level 0 file that is data only  

In [75]:
#If the metadata exists, join it to the data and write out as a level 1 file
metadata_filename = os.path.join(data_path,plateID,"metadata",plateID+".xlsx")

if os.path.exists(metadata_filename):
    md_all = pd.read_excel(metadata_filename, engine='openpyxl', dtype={'Drug1Concentration': str, 'Drug2Concentration': str})
    
    #remove unwanted columns read in from the excel files
    r = re.compile("Unnamed.*")
    columns_to_drop = list(filter(r.match, md_all.columns)) 
    metadata = md_all.drop(columns = columns_to_drop)
    
    #match metadata and data well labels format
    metadata['row'] = [re.sub(r'[0-9]*', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'[A-Z]', '', Well) for Well in metadata['Well']]
    metadata['column'] = [re.sub(r'\A0', '', row) for row in metadata['column']]
    metadata['well'] = metadata['row'] + metadata['column']
    

#### Pull data from images  
Apply the filtered masks to the registered cytoplasmic channel and record each cell's features in total cell, cytoplasm alone, nuceli and nuclear ring 
If the metadata is available, merge it with the cyoplasmic data   
Store the cell level data (and metadata) in a csv file where each row is a cell  
Data feature values can be decoded as follows:  
\<compartment>\_\<pipeline name>\_\<channel name>\_\<regionprops name>  
compartment - Cell, Cyto, Nuclei or NucExp 
pipeline name - CK for cellpose KIT or other if added  
channel name - NR for nuclear reporter, CC for cell cycle reporter or others if added  
regionprops name - label passed through from the skimage measure.regionprops function https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops  



In [76]:
minutes_between_images = 30

#loop through the fields in the well

for subdir in subdirectories:
    field = re.findall("field_[1-9]",subdir)[0]
    l0_filename = os.path.join(data_path+plateID,"Analysis",pipeline_name,plateID+"_"+well+"_"+field+"_cell_level_0.csv")
    if not (os.path.exists(l0_filename) | os.path.exists(l0_filename.replace('level_0','level_1'))): 
        filtered_mask_path = os.path.join(output_path,"tracking",well,field,"filtered_masks")
        tracked_mask_filenames = sorted(glob.glob(filtered_mask_path+"/mask*"))
        img_ps_reg = io.imread(output_path+plateID+"_P_"+well+"_"+field.replace("field_","")+"_reg_stack.tif")
        img_rs_reg = io.imread(output_path+plateID+"_R_"+well+"_"+field.replace("field_","")+"_reg_stack.tif")
        img_gs_reg = io.imread(output_path+plateID+"_G_"+well+"_"+field.replace("field_","")+"_reg_stack.tif")
        # iterate over the cyto mask files
        all_results = []
        for i, fn in enumerate(tracked_mask_filenames):
            #read in the cytoplasm mask image
            masks = io.imread(fn)

            #measure cell cycle reporter intensity and cellular morphology, texture
            cell_cell_cycle = measure.regionprops_table(masks, intensity_image=img_gs_reg[i],
                                               properties=('label',
                                                           'area','bbox_area','convex_area','centroid','eccentricity','equivalent_diameter','extent','feret_diameter_max','filled_area',
                                                            'major_axis_length','minor_axis_length','moments_hu','perimeter','perimeter_crofton','solidity',
                                                            'mean_intensity','max_intensity','min_intensity'))

            # turn results into a dataframe
            cell_data = pd.DataFrame(cell_cell_cycle)
            cell_data.rename(columns={col: 'Cell_'+pipeline_name+'_' +ch2_name+'_'+col  for col in cell_data.columns if col not in ['label']}, inplace=True)

            # recover the well, field and time slice values and add them to the dataframe
            well = re.findall('/[A-Z][0-9]+/',fn)[0]
            well = re.sub('/','', well)
            cell_data['well'] = well
            field = re.findall('field_[0-9]+',fn)[0]
            field = int(re.sub('field_','', field))
            cell_data['field'] = field
            cell_data['slice'] = i
            elapsed_minutes = i*minutes_between_images #assumes time slice numbering starts at 1
            day = np.floor(elapsed_minutes/(24*60)).astype(int)
            hour = np.floor((elapsed_minutes-day*(24*60))/60).astype(int)
            minute = np.floor(elapsed_minutes-day*(24*60)-hour*60).astype(int)
            day = str(day).zfill(2)
            hour = str(hour).zfill(2)
            minute = str(minute).zfill(2)
            cell_data['time_slice'] = day+"d"+hour+"h"+minute+"m"

            #read in the nuclear mask image
            nfn = fn.replace("filtered_masks","nuc_masks")
            nfn = nfn.replace("/mask","/nuc_mask")
            nuc_masks = io.imread(nfn)

            #measure cell cycle reporter intensity and nuclear morphology, texture
            nuc_cell_cycle = measure.regionprops_table(nuc_masks, intensity_image=img_gs_reg[i],
                                               properties=('label',
                                                           'area','bbox_area','convex_area','centroid','eccentricity','equivalent_diameter','extent','feret_diameter_max','filled_area',
                                                            'major_axis_length','minor_axis_length','moments_hu','perimeter','perimeter_crofton','solidity',
                                                            'mean_intensity','max_intensity','min_intensity'))

            # turn results into a dataframe
            nuc_data = pd.DataFrame(nuc_cell_cycle)
            nuc_data.rename(columns={col: 'Nuclei_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuc_data.columns if col not in ['label']}, inplace=True)
            nuc_data.rename(columns={"label": "nuc_label"}, inplace=True)

            #compute the distances between the cyto and nuclei centroids
            #Rows are cyto and columns are nuclei
            #smallest value in each row is the nearest nuclei to the cyto mask's centroid
            centroid_distances = pd.DataFrame(spatial.distance_matrix(cell_data[['Cell_CK_CC_centroid-1', 'Cell_CK_CC_centroid-0']].to_numpy(),
            nuc_data[['Nuclei_CK_CC_centroid-1', 'Nuclei_CK_CC_centroid-0']].to_numpy()))

            #for each row, find the column with the smallest value
            nearest_nuclei = centroid_distances.idxmin(axis = 1)

            #add the nuclear label to the cyto data
            cell_data['nuclei_label'] = nearest_nuclei
            
            #add the nuclear expansion ring data to the nuclei data
            #read in the nuclear expansion mask image
            nefn = fn.replace("filtered_masks","nuc_exp_masks")
            nefn = nfn.replace("/mask","/nuc_exp_mask")
            nuc_exp_masks = io.imread(nefn)

            #measure cell cycle reporter intensity and nuclear morphology, texture
            nuc_exp_cell_cycle = measure.regionprops_table(nuc_exp_masks, intensity_image=img_gs_reg[i],
                                               properties=('label','mean_intensity','max_intensity','min_intensity'))

            # turn results into a dataframe
            nuc_exp_data = pd.DataFrame(nuc_exp_cell_cycle)
            nuc_exp_data.rename(columns={col: 'NucExp_'+pipeline_name+'_' +ch2_name+'_'+col  for col in nuc_exp_data.columns if col not in ['label']}, inplace=True)
            nuc_exp_data.rename(columns={"label": "nuc_exp_label"}, inplace=True)

            #add the cytoplasm data to the cell data
            #read in the cytoplasm mask image
            cyfn = fn.replace("filtered_masks","cyto_masks")
            cyfn = cyfn.replace("/mask","/cyto_mask")
            cyto_masks = io.imread(cyfn)

            #measure cell cycle reporter intensity in the cytoplasm
            cyto_cell_cycle = measure.regionprops_table(cyto_masks, intensity_image=img_gs_reg[i],
                                               properties=('label','mean_intensity','max_intensity','min_intensity'))

            # turn results into a dataframe
            cyto_data = pd.DataFrame(cyto_cell_cycle)
            cyto_data.rename(columns={col: 'Cyto_'+pipeline_name+'_' +ch2_name+'_'+col  for col in cyto_data.columns if col not in ['label']}, inplace=True)
            cyto_data.rename(columns={"label": "cyto_label"}, inplace=True)
            
            #merge cell and cyto data on label
            cell_cyto_data = pd.merge(cell_data, cyto_data, left_on='label', right_on='cyto_label')
            
            #merge nuclei and nuclei ring data on label
            nuc_nuc_exp_data = pd.merge(nuc_data, nuc_exp_data, left_on='nuc_label', right_on='nuc_exp_label')
            
            #merge the cyto and nuclear data on the label
            all_data = pd.merge(cell_cyto_data, nuc_nuc_exp_data, left_on='nuclei_label', right_on='nuc_label')
            
            #Calculate ratio of ch2 cyto to nuclei intensities
            all_data['Cell_'+pipeline_name+'_' +ch2_name+'_mean_intensity_ratio'] = all_data['Cyto_'+pipeline_name+'_' +ch2_name+'_mean_intensity']/all_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_mean_intensity']
            all_data['Cell_'+pipeline_name+'_' +ch2_name+'_max_intensity_ratio'] = all_data['Cyto_'+pipeline_name+'_' +ch2_name+'_max_intensity']/all_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_max_intensity']
            all_data['Cell_'+pipeline_name+'_' +ch2_name+'_min_intensity_ratio'] = all_data['Cyto_'+pipeline_name+'_' +ch2_name+'_min_intensity']/all_data['Nuclei_'+pipeline_name+'_' +ch2_name+'_min_intensity']

            # append the dataframe to the results list
            all_results.append(all_data)

        #concatenate all of the results from the images in the field
        l0 = pd.concat(all_results)

        #join with the tracking results to get lineage, parent, frame length values
        tracks_filename = os.path.join(output_path,"tracking",well,"field_"+str(field),"results/tracks.csv")
        tracks = pd.read_csv(tracks_filename)
        l0 = pd.merge(l0, tracks, how="left", on=["label", "well", "field"])

        if os.path.exists(metadata_filename):
            #merge data and metadata on well values
            l1= pd.merge(l0, metadata, how="left", on=["well"]).round(decimals=2)
            l1.to_csv(l0_filename.replace('level_0','level_1'), index = False)
        else:
            print("no metadata file for "+plateID+" so creating level 0 file")
            l0 = l0.round(decimals=2)
            l0.to_csv(l0_filename, index = False)
    